# **ТЗ для вакансии “Data Scientist Intern”**


**1. Найти стоимость тарифа доставки для каждого склада**

---


Для этого считаем json-файл с данными о заказах, затем для каждого заказа будем вычислять отношение суммарной стоимости доставки всех продуктов к количеству проданного товара. Полученные данные обернем в множество, состоящее из пар: наименование склада и стоимость его тарифа доставки.  

In [1]:
import json
import pandas as pd
import numpy as np

from google.colab import drive


# Cчитаем данные из json-файла
drive.mount('/content/drive')

with open('/content/drive/My Drive/trial task/trial_task.json') as json_file:
    data = json.load(json_file)

'''with open('trial_task.json') as json_file:
  data = json.load(json_file)'''


Mounted at /content/drive


"with open('trial_task.json') as json_file:\n  data = json.load(json_file)"

In [2]:
''' Для каждого заказа будем вычислять отношение суммарной стоимости
    доставки всех продуктов к количеству проданного товара '''

delivery_cost_rate_for_each_warehouse = set()

for i in range(len(data)):
    total_quantity_for_each_warehouse = 0

    for product in data[i]['products']:
        total_quantity_for_each_warehouse += product['quantity']

    delivery_cost_rate_for_each_warehouse.add((data[i]['warehouse_name'],
                                             int(data[i]['highway_cost'] /
                                             total_quantity_for_each_warehouse)))

In [3]:
# Из полученного множества создадим DataFrame
delivery_cost_rate_df = pd.DataFrame(
    data = delivery_cost_rate_for_each_warehouse,
    columns = ['warehouse_name', 'delivery_cost_rate'])

delivery_cost_rate_df.head()

,warehouse_name,delivery_cost_rate
0,отель Лето,-25
1,хутор близ Диканьки,-15
2,Мордор,-10
3,остров невезения,-5
4,гиперборея,-20


**2.  Найти суммарное количество , суммарный доход , суммарный расход и суммарную прибыль для каждого товара (представить как таблицу со столбцами
'product', 'quantity', 'income', 'expenses', 'profit')**


---




In [4]:
# Создаём пустой DataFrame с заданными столбцами
columns = [
    'product',
    'quantity',
    'income',
    'expenses',
    'profit'
]

sum_indicators_for_each_product = pd.DataFrame(columns=columns)

In [5]:
# Определяем, сколько существует уникальных товаров
products_set = set()

for i in range(len(data)):
    for product in data[i]['products']:
        products_set.add(product['product'])

products_set

{'автограф Стаса Барецкого',
 'билет в Израиль',
 'зеленая пластинка',
 'ломтик июльского неба',
 'плюмбус',
 'подписка на suppi-блог',
 'статуэтка Ленина'}

In [7]:
# Занесём данные о продуктах в таблицу, а остальное инициализируем нулями
sum_indicators_for_each_product['product'] = list(products_set)

for column in columns[1:]:
    sum_indicators_for_each_product[column] = np.zeros(len(products_set))

sum_indicators_for_each_product.head(10)

,product,quantity,income,expenses,profit
0,билет в Израиль,0.0,0.0,0.0,0.0
1,плюмбус,0.0,0.0,0.0,0.0
2,подписка на suppi-блог,0.0,0.0,0.0,0.0
3,статуэтка Ленина,0.0,0.0,0.0,0.0
4,зеленая пластинка,0.0,0.0,0.0,0.0
5,автограф Стаса Барецкого,0.0,0.0,0.0,0.0
6,ломтик июльского неба,0.0,0.0,0.0,0.0


Опишем функции для вычисления основных суммарных показателей для продукта

In [8]:
# Вычисление суммарного количества переданного продукта
def compute_sum_quantity_for_product(product_name):

    quantity = 0

    for i in range(len(data)):
        for product in data[i]['products']:

            if product_name == product['product']:
                quantity += product['quantity']

    return quantity

In [9]:
# Вычисление суммарного дохода с продукта
def compute_sum_income_for_product(product_name):

    sum_income = 0

    for i in range(len(data)):
        for product in data[i]['products']:

            if product_name == product['product']:
                sum_income += product['price'] * product['quantity']

    return sum_income

In [10]:
# Вычисление суммарных расходов на продукт
def compute_sum_expenses_for_product(product_name):

    sum_expenses = 0

    for i in range(len(data)):
        warehouse_name = data[i]['warehouse_name']

        index = delivery_cost_rate_df.index[
            delivery_cost_rate_df['warehouse_name'] == warehouse_name]

        delivery_rate = int(
            delivery_cost_rate_df.iloc[index]['delivery_cost_rate'])


        for product in data[i]['products']:
            if product_name == product['product']:
                sum_expenses += product['quantity'] * delivery_rate


    return sum_expenses

In [11]:
# Вычисление суммарной прибыли для продукта
def compute_sum_profit_for_product(product_name):

    income = compute_sum_income_for_product(product_name)
    expenses = compute_sum_expenses_for_product(product_name)

    return income + expenses

In [12]:
# Заполнение суммарного показателя для каждого продукта
def fill_in_the_product(compute):

 # В качестве аргумента передаем функцию,
 # вычисляющую определенный суммарный показатель

    product_indicators = []

    for index, product_name in enumerate(
                                   sum_indicators_for_each_product['product']):

        product_indicators.append(compute(product_name))

    return product_indicators

Заполним столбцы таблицы sum_indicators_for_each_product


In [37]:
# Заполняем столбцы таблицы sum_indicators_for_each_product

sum_indicators_for_each_product['quantity'] = fill_in_the_product(
    compute_sum_quantity_for_product)

sum_indicators_for_each_product['income'] = fill_in_the_product(
    compute_sum_income_for_product)

sum_indicators_for_each_product['expenses'] = fill_in_the_product(
    compute_sum_expenses_for_product)

sum_indicators_for_each_product['profit'] = fill_in_the_product(
    compute_sum_profit_for_product)

sum_indicators_for_each_product.head(7)

,product,quantity,income,expenses,profit
0,билет в Израиль,58,58000,-1050,56950
1,плюмбус,65,16250,-940,15310
2,подписка на suppi-блог,33,4950,-545,4405
3,статуэтка Ленина,68,13600,-935,12665
4,зеленая пластинка,61,610,-920,-310
5,автограф Стаса Барецкого,48,28800,-820,27980
6,ломтик июльского неба,42,18900,-670,18230


**3. Составить табличку со столбцами 'order_id' (id заказа) и 'order_profit' (прибыль полученная с заказа). А также вывести среднюю прибыль заказов**

---




In [38]:
orders_profit = pd.DataFrame(columns = ['order_id', 'order_profit'])

order_id = []
order_profit = []

# В цикле обходим все заказы и заполняем два списка: order_id и order_profit
for i in range(len(data)):
    income_from_the_order = 0
    expenses_from_the_order = 0
    highway_cost = data[i]['highway_cost']

    order_id.append(data[i]['order_id'])

    for product in data[i]['products']:
        income_from_the_order += product['price'] * product['quantity']

    order_profit.append(income_from_the_order + highway_cost)


In [39]:
# Проверим, совпадают ли длины списков
assert len(order_id) == len(order_profit)

# Заполним наш DataFrame orders_profit
orders_profit['order_id'] = order_id
orders_profit['order_profit'] = order_profit

In [40]:
# Вычислим среднюю прибыль заказов,
# выведем её на экран вместе с частью таблицы order_profit

mean_order_profit = orders_profit['order_profit'].mean()

print(f'mean order profit: {mean_order_profit}')
orders_profit.head(10)

mean order profit: 1352.3


,order_id,order_profit
0,11973,3980
1,62239,985
2,85794,-30
3,33684,1980
4,25824,975
5,87044,735
6,58598,1590
7,5430,1120
8,60502,1725
9,96473,0


**4. Составить табличку типа 'warehouse_name' , 'product','quantity', 'profit', 'percent_profit_product_of_warehouse' (процент прибыли продукта заказанного из определенного склада к прибыли этого склада)**


---



In [47]:
warehouses_df = pd.DataFrame(columns = ['warehouse_name',
                                        'product',
                                        'quantity',
                                        'profit',
                                        'percent_profit_product_of_warehouse'])
war
for i in range(len(data)):
    if data[i]['warehouse_name']
    delivery_cost_rate_df['warehouse_name']


,warehouse_name,product,quantity,profit,percent_profit_product_of_warehouse
0,отель Лето,NaN,NaN,NaN,NaN
1,хутор близ Диканьки,NaN,NaN,NaN,NaN
2,Мордор,NaN,NaN,NaN,NaN
3,остров невезения,NaN,NaN,NaN,NaN
4,гиперборея,NaN,NaN,NaN,NaN
